In [1]:
import weaviate
import json
import arxiv
import os
from datetime import datetime
from tqdm import tqdm
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index import VectorStoreIndex, ServiceContext
from llama_index import SimpleDirectoryReader
from llama_index import Document
from llama_index.vector_stores import WeaviateVectorStore
from llama_index import VectorStoreIndex, StorageContext
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
from llama_index.postprocessor import MetadataReplacementPostProcessor, SentenceTransformerRerank
from llama_index.response.notebook_utils import display_response
from llama_index.readers.file.docs_reader import PDFReader
from llama_index import load_index_from_storage

c:\Users\sumuk\anaconda3\envs\ResearchSurveyLLM\lib\site-packages\llama_index\vector_stores\lancedb.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  from pandas import DataFrame
c:\Users\sumuk\anaconda3\envs\ResearchSurveyLLM\lib\site-packages\llama_index\download\module.py:12: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [2]:
start_date = "20220101000000" 
end_date = "20240131235959"    
topic = "Denoising Diffusion models"
max_count = 5
required_metadata = ["authors", "title", "summary", "published"]
temp_save_folder = r"C:\Users\sumuk\OneDrive\Desktop\GitHub\ResearchSurveyLLM\delete"

In [3]:
def get_metadata(paper_object):
    metadata = dict()
    for i in required_metadata:
        if i == "authors":
            authors = ", ".join([j.name for j in getattr(paper_object, i)])
            metadata[i] = authors
        elif i == "published":
            metadata[i] = paper_object.published.ctime()
        else:
            metadata[i] = str(getattr(paper_object, i)).replace("\n", " ")
    return metadata

In [4]:

client = arxiv.Client(num_retries=10, page_size=500)
search_query = arxiv.Search(query = topic,
                            sort_by = arxiv.SortCriterion.Relevance
                            )
start_date = datetime.strptime(start_date, "%Y%m%d%H%M%S")
end_date = datetime.strptime(end_date, "%Y%m%d%H%M%S")
results = client.results(search_query)

In [6]:
paper_arxiv_objects = []
current_paper_count = 0
for r in tqdm(results):
    if start_date.timestamp() <= r.published.timestamp() <= end_date.timestamp():
        current_paper_count += 1
        # print(r.title)
        r.download_pdf(temp_save_folder, f"paper_{current_paper_count}" + ".pdf")
        paper = PDFReader().load_data(os.path.join(temp_save_folder, f"paper_{current_paper_count}" + ".pdf"))
        paper = Document(text="\n\n".join([doc.text for doc in paper]), metadata=get_metadata(r))
        paper_arxiv_objects.append(paper)        
        if current_paper_count == max_count:
            break

5it [00:43,  8.74s/it]


In [7]:
client = weaviate.Client(
    url = "http://localhost:8080"
)

c:\Users\sumuk\anaconda3\envs\ResearchSurveyLLM\lib\site-packages\weaviate\warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


In [8]:
schema = {
            "classes": [
                {
                "class": "ArXivPaper",
                "description": "Represents a paper from ArXiv",
                "properties": [
                    {
                    "name": "title",
                    "dataType": ["string"],
                    "description": "Title of the paper"
                    },
                    {
                    "name": "authors",
                    "dataType": ["string"],
                    "description": "Authors of the paper"
                    },
                    {
                    "name": "abstract",
                    "dataType": ["string"],
                    "description": "Abstract of the paper"
                    },
                    {
                    "name": "content",
                    "dataType": ["text"],
                    "description": "Full text content of the paper"
                    },
                    {
                    "name": "url",
                    "dataType": ["string"],
                    "description": "URL of the paper on ArXiv"
                    },
                    {
                    "name": "published_date",
                    "dataType": ["date"],
                    "description": "Date when the paper was published on ArXiv"
                    }
                ]
                }
            ]
         }
client.schema.create(schema)

UnexpectedStatusCodeError: Create class! Unexpected status code: 422, with response body: {'error': [{'message': 'class name "ArXivPaper" already exists'}]}.

In [15]:
def get_build_index(documents, embed_model=r"local:C:\Users\sumuk\OneDrive\Desktop\GitHub\ResearchSurveyLLM\UAE-Large-V1", sentence_window_size=3,save_dir="C:/Users/sumuk/OneDrive/Desktop/GitHub/ResearchSurveyLLM/delete/index"):
  node_parser = SentenceWindowNodeParser(
      window_size = sentence_window_size,
      window_metadata_key = "window",
      original_text_metadata_key = "text"
  )

  vector_store = WeaviateVectorStore(weaviate_client = client, index_name="ArXivPaper", text_key="content")
  storage_context = StorageContext.from_defaults(vector_store = vector_store)


  sentence_context = ServiceContext.from_defaults(
      llm = None,
      embed_model= embed_model,
      node_parser = node_parser,
  )

  if not os.path.exists(save_dir):
        # create and load the index
        index = VectorStoreIndex.from_documents(
            documents, service_context=sentence_context, storage_context = storage_context
        )
        index.storage_context.persist(persist_dir=save_dir)
  else:
      # load the existing index
      index = load_index_from_storage(
          StorageContext.from_defaults(persist_dir=save_dir),
          service_context=sentence_context,
      )

  return index

In [16]:
paper_arxiv_objects[0]

Document(id_='789bd353-59b8-4650-9469-f6b296c4efa5', embedding=None, metadata={'authors': 'Zhennan Chen, Rongrong Gao, Tian-Zhu Xiang, Fan Lin', 'title': 'Diffusion Model for Camouflaged Object Detection', 'summary': 'Camouflaged object detection is a challenging task that aims to identify objects that are highly similar to their background. Due to the powerful noise-to-image denoising capability of denoising diffusion models, in this paper, we propose a diffusion-based framework for camouflaged object detection, termed diffCOD, a new framework that considers the camouflaged object segmentation task as a denoising diffusion process from noisy masks to object masks. Specifically, the object mask diffuses from the ground-truth masks to a random distribution, and the designed model learns to reverse this noising process. To strengthen the denoising learning, the input image prior is encoded and integrated into the denoising diffusion model to guide the diffusion process. Furthermore, we d

In [17]:
index = get_build_index(paper_arxiv_objects)

c:\Users\sumuk\anaconda3\envs\ResearchSurveyLLM\lib\site-packages\weaviate\__init__.py:128: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Please import it from its specific module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])


LLM is explicitly disabled. Using MockLLM.


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
